# Import packages

### Import python packages

In [ ]:
import re,os,glob
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output
import pandas as pd
import math
import random
from datetime import datetime
from scipy.stats import linregress
from sklearn.decomposition import PCA
from tqdm import trange, tqdm
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import scipy
from scipy import interpolate
tfk = tf.keras
tfkl = tf.keras.layers
clear_output()
from MRA_generate import MRA_generate
from parameters import *
import symae_core as symae
from movie_redatum import movie_redatum 

2022-08-08 21:57:45.522183: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Generate dataset X

### Define $n_\varepsilon$ states

In [ ]:
def g(n,x):
    if n==0:
        return int(x<0.5)
    elif n==1:
        return math.cos(2*math.pi*x)/2 + 0.5
    elif n==2:
        if x<0.5: 
            return 1-2*x
        else:     
            return 0
    elif n==3:
        return 2*int(x<0.5)-1
    elif n==4:
        return math.cos(2*math.pi*x)
    elif n==5:
        if x<0.5: 
            return 1-4*x
        else:     
            return -1
#def g(n,x):
#    return math.e**(-(n+1)*(x-0.5)**2)

In [ ]:
fig, axs = plt.subplots(1, ne, sharex=True, sharey=True, figsize=(ne*4, 4))
for i in range(ne):
    axs[i].plot(range(d), [g(i,x/d) for x in range(d)])
plt.suptitle('Show the shape of all $n_\epsilon$ = %d states'%ne)

### Generate the dataset

In [ ]:
MRA_training = MRA_generate(d,nt,N,0,ne,g,replace=0,continuous=False)
MRA_training.generate_default()
X = MRA_training.X
states = MRA_training.states
shifts = MRA_training.shifts

### Print the structure of dataset X

In [ ]:
select_times = MRA_training.select_times
plt.figure(figsize=(5,5),dpi=100)
plt.matshow(select_times.T, aspect='auto', fignum=1)
plt.colorbar()
plt.title('How many times has each block in D been selected by X?')
plt.show()

# Train SymAE

In [ ]:
nt

### Layer

In [ ]:
class LatentCat(tf.keras.Model):
    def __init__(self, alpha=0.5):
        super(LatentCat, self).__init__(name = 'latentcat')
        self.drop = tfkl.Dropout(alpha)
    def call(self, zsym, znuisance, training=False):
        znuisance = self.drop(znuisance, training=training)
        znuisance = tfkl.Flatten()(znuisance)
        z = tfkl.concatenate([zsym, znuisance])
        return z

In [ ]:
class SymAE(tf.keras.Model):
    def __init__(self, N, nt, d, p, q, r): 
        super(SymAE, self).__init__()
        # Build symmetric encoder
        sym_encoder = symae.SymmetricEncoder1D(p)
        self.sym_encoder = sym_encoder
        
        # Build nuisance encoder
        nui_encoder = symae.NuisanceEncoder1D(q)
        self.nui_encoder = nui_encoder
        
        # Build distribute in decoder
        distzsym = symae.DistributeZsym(nt, p, q)
        self.distzsym = distzsym
        
        #Build mixer in decoder
        mixer = symae.Mixer1D(d)
        self.mixer = mixer
        
        # Build latentcat
        latentcat = LatentCat(r)
        self.latentcat = latentcat        
        
        # Build encoder
        encoder_input = tfk.Input(shape=(nt,d,1), dtype='float32', name='encoder_input')
        znuisance = nui_encoder(encoder_input)
        zsym = sym_encoder(encoder_input)
        encoder_output = latentcat(zsym,znuisance)
        encoder = tfk.Model(encoder_input, encoder_output, name="encoder")
        self.encoder = encoder
        
        # Build decoder
        decoder_input = tfk.Input(shape=(p+q*nt), name='latentcode')
        decoder_output = mixer(distzsym(decoder_input))
        decoder = tfk.Model(decoder_input,decoder_output, name="decoder") 
        self.decoder = decoder
        
    def call(self, x):
        sym_code = self.sym_encoder(x)
        nui_code = self.nui_encoder(x)
        z = self.latentcat(sym_code, nui_code)
        dist = self.distzsym(z)
        out = self.mixer(dist)
        return out

### Initialize SymAE and load weights

In [ ]:
my_list=[(5, 10, 0.5, 0)]

In [ ]:
for (p,q,r,lam) in my_list:
    path = './add,p=%d,q=%d,r=%.1f,l=%.2f'%(p,q,r,lam)    
    #import symae_model 
    model = SymAE(N,nt,d,p,q,r)
    #model = symae_model.SymAE(N,nt,d,p,q,5,64,0.0)
    clear_output()
    #lam=10
    try: 
        model.load_weights(path+'/checkpoint')
        print("old")    
    except:
        print("new")
    #lam=10
    optimizer = tf.keras.optimizers.Adam(0.001)
    mse = tf.keras.losses.MeanSquaredError()

    def reconstruction_loss(model, x, training=False):
        x_hat = model(x, training=True)
        return mse(x, x_hat)
    
    @tf.function
    def train_step(model, x, training=True):
        with tf.GradientTape() as tape:
            loss_rec = reconstruction_loss(model, x, training)
            loss = loss_rec
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss_rec

    try:
        os.mkdir(path)
    except:
        print('exists')

    try:
        epoch = int(re.findall(r"\d+",glob.glob(path+'/*.epoch')[0])[-1])
        print(epoch)
    except:
        epoch = 0
        os.mkdir(path+'/0.epoch')

    losses=[]
    for i in range(1000):
        epoch += 1
        os.mkdir(path+'/%d.epoch'%epoch)
        os.rmdir(path+'/%d.epoch'%(epoch-1))    
        loss_rec = train_step(model, X)
        print("For epoch {:d}, reconstruction loss is {:f}."
              .format(epoch, loss_rec))
        clear_output(wait=True)
        losses.append(loss_rec.numpy())
    

In [ ]:
plt.plot(range(len(losses)),losses)

In [ ]:
MRA_test = MRA_generate(d,nt,100,sigma,ne,g,replace=0,continuous=False)
MRA_test.generate_default()
clear_output()
N_states = np.repeat(MRA_test.states,nt)
N_shifts = MRA_test.shifts.reshape(-1,1)
df_states = pd.DataFrame(1+N_states,columns=['state']).astype('object')
df_shifts = pd.DataFrame(N_shifts,columns=['shift'])

In [ ]:
N_code = model.nui_encoder.predict(MRA_test.X,verbose=0)
N_reshaped = N_code.reshape(-1,q)
pca_N = PCA(2).fit_transform(N_reshaped)
df = pd.DataFrame(pca_N, columns = ['first principal component','second principal component'])
df = pd.concat([df,df_states,df_shifts],axis=1)
df = df.sort_values(by=['state'])
fig = px.scatter(df, x='first principal component', y='second principal component', 
                 color='state', text='shift')
fig.update_layout(width=500, height=400, title="PCA of latent nuisance codes")
fig.show()
fig.write_image(path+'/latent.pdf')

In [ ]:
fig = movie_redatum(g,model)
fig.savefig(path+"/redatum.pdf", format='pdf', dpi=200)